In [ ]:
# !pip install snowflake-snowpark-python[pandas]==1.5.0
# !pip install snowflake-connector-python
# !pip install snowflake-ml-python==1.0.10

In [ ]:
!pip install snowflake[ml]

In [ ]:
!pip install snowflake-connector-python


In [2]:
from snowflake.snowpark import Session

In [3]:
import configparser
def get_session():
    parser = configparser.ConfigParser()
    # Add the credential file name here
    parser.read('/notebooks/notebooks/config.ini')

    connection_params = dict(user=parser['Credentials']['user'], 
                         password=parser['Credentials']['password'], 
                         account=parser['Credentials']['account'], 
                         warehouse=parser['Credentials']['warehouse'], 
                         database=parser['Credentials']['database'],
                         schema=parser['Credentials']['schema'], 
                         role=parser['Credentials']['role'])
    print("connection_params = ",  connection_params)


    session = Session.builder.configs(connection_params).create()
    return session

In [4]:
session = get_session()
# session.close()
session

connection_params =  {'user': 'PRAKHAR', 'password': 'Prak@apr24', 'account': 'ug94937.us-east4.gcp', 'warehouse': 'FOSFOR_INSIGHT_WH', 'database': 'test_database_prak', 'schema': 'test_schema_prak', 'role': 'PRAKHAR'}


In [5]:
REGISTRY_DATABASE_NAME = "MODEL_REGISTRY_SNOWSIGHT"
REGISTRY_SCHEMA_NAME = "SNOWSIGHT"

In [1]:
from snowflake.ml.registry import model_registry

In [ ]:
!pip freeze | grep -i cloud

In [6]:
model_registry.create_model_registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

create_model_registry() is in private preview since 0.2.0. Do not use it in production. 


True

In [10]:
registry1 = model_registry.Registry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME )

AttributeError: module 'snowflake.ml.registry.model_registry' has no attribute 'Registry'

In [11]:
registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)

<ipython-input-11-ac16ff576944>:1: DeprecationWarning: 
The `snowflake.ml.registry.model_registry.ModelRegistry` has been deprecated starting from version 1.2.0.
It will stay in the Private Preview phase. For future implementations, kindly utilize `snowflake.ml.registry.Registry`,
except when specifically required. The old model registry will be removed once all its primary functionalities are
fully integrated into the new registry.
        
  registry = model_registry.ModelRegistry(session=session, database_name=REGISTRY_DATABASE_NAME, schema_name=REGISTRY_SCHEMA_NAME)


In [12]:
registry.get_history().show()

-------------------------------------------------------------------------------------------------------------------
|"EVENT_TIMESTAMP"  |"EVENT_ID"  |"MODEL_ID"  |"ROLE"  |"OPERATION"  |"ATTRIBUTE_NAME"  |"VALUE[ATTRIBUTE_NAME]"  |
-------------------------------------------------------------------------------------------------------------------
|                   |            |            |        |             |                  |                         |
-------------------------------------------------------------------------------------------------------------------



In [14]:
import cloudpickle

model_obj = cloudpickle.load(open("/notebooks/notebooks/ml_model", "rb"))
model_obj

array(['age', 'height_cm', 'weight_kg', 'skill_dribbling',
       'attacking_crossing', 'attacking_finishing',
       'movement_acceleration', 'movement_sprint_speed',
       'power_shot_power', 'mentality_aggression', 'defending_marking',
       'defending_standing_tackle', 'goalkeeping_diving',
       'goalkeeping_handling', 'overall', 'potential'], dtype=object)

In [15]:
x_train = cloudpickle.load(open("/notebooks/notebooks/x_train", "rb"))
x_train

,age,height_cm,weight_kg,skill_dribbling,attacking_crossing,attacking_finishing,movement_acceleration,movement_sprint_speed,power_shot_power,mentality_aggression,defending_marking,defending_standing_tackle,goalkeeping_diving,goalkeeping_handling,overall,potential
620,25,185,88,78,65,81,77,75,81,63,37,26,9,10,79,82
2193,26,184,77,64,47,41,65,64,56,86,73,75,14,14,74,77
1170,32,182,84,85,61,80,75,67,77,51,25,30,6,16,77,77
1863,34,183,73,73,63,79,76,49,79,51,57,19,8,10,75,75
4724,24,176,70,71,66,63,74,77,70,61,64,71,10,8,70,76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3671,32,195,85,12,14,12,35,43,46,22,16,14,74,68,72,72
709,27,184,78,73,78,61,69,72,84,75,79,79,14,5,79,79
2487,29,180,71,73,64,72,55,52,70,82,47,64,10,15,74,74
174,23,171,69,84,84,50,86,84,79,68,75,78,7,13,83,87


In [16]:
mv = registry.log_model(model_obj,
                   model_name="my_model",
                   version_name="v1",
                   conda_dependencies=["scikit-learn"],
                   comment="My awesome ML model",
#                    metrics={"score": 96},
                   sample_input_data=x_train)

TypeError: (0000) log_model() got multiple values for argument 'model_name'